In [ ]:
rawRDD = sc.textFile('data/covtype.data.gz')

In [ ]:
column_names = (
["Elevation",
"Aspect",
"Slope",
"Horizontal_Distance_To_Hydrology",
"Vertical_Distance_To_Hydrology",
"Horizontal_Distance_To_Roadways",
"Hillshade_9am",
"Hillshade_Noon",
"Hillshade_3pm",
"Horizontal_Distance_To_Fire_Points"]
    + ['WE{}'.format(i) for i in range(4)]
    + ['ST{}'.format(i) for i in range(40)]
    + ['Cover'])

In [ ]:
from pyspark.sql import Row

In [ ]:
cover = spark.createDataFrame(
    rawRDD
     .map(lambda x: [float(i) for i in x.split(',')])
     .map(lambda x: Row(**{col:v for col, v in zip(column_names, x)})))

In [ ]:
cover.printSchema()

In [ ]:
cover['elevation', 'aspect', 'slope'].show(5)

In [ ]:
cover.select(cover['elevation'], cover['slope']).show(5)

In [ ]:
cover.select(cover['elevation'] / 10).show(5)

In [ ]:
cover.filter(cover['slope']>10).select(cover['elevation'], cover['slope']).show(5)

In [ ]:
d = cover.sample(True, 0.05)

In [ ]:
d.cache()

In [ ]:
d.groupBy('Cover').agg({'slope': 'avg'}).show()

In [ ]:
d.groupBy('Cover').agg({'slope': 'avg', 'slope': 'min', 'aspect': 'max'}).show()

In [ ]:
d.groupBy('Cover').agg({'slope': 'avg', 'slope': 'min', 'aspect': 'max'}).toPandas()

In [ ]:
d.groupBy('Cover').agg({'slope': 'avg', 'slope': 'min', 'aspect': 'max'}).toJSON().collect()

In [ ]:
d.cov('Slope', 'Elevation')

In [ ]:
d.approxQuantile('Elevation', [0.25, 0.5, 0.75], 0.1)

In [ ]:
d.corr('Slope', 'Elevation')

In [ ]:
d.describe('Slope').show()

In [ ]:
dna = spark.createDataFrame(
    sc.parallelize([Row(name='John', age=51), Row(name='Jim', age=None), Row(name=None, age=56)]))

In [ ]:
dna.show()

In [ ]:
dna.dropna().show()

In [ ]:
dna.na.fill({'name': 'unkown', 'age': 99}).show()

In [ ]:
d.freqItems(['Slope', 'Elevation']).show()

In [ ]:
from pyspark.sql import functions as fn

In [ ]:
d.agg(fn.approxCountDistinct(d.Slope).alias('dSlope')).show()

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
feature_maker = VectorAssembler(inputCols=column_names[:-1], outputCol='features')

In [ ]:
data = feature_maker.transform(cover)

In [ ]:
train, test = data.randomSplit([0.7, 0.3])

In [ ]:
model = LogisticRegression(labelCol='Cover').fit(train)

In [ ]:
coefs = model.coefficientMatrix.toArray()

In [ ]:
result = model.evaluate(test.sample(False, 0.05))

In [ ]:
result.predictions.select((result.predictions.Cover == result.predictions.prediction).cast('float').alias('hit')
                         ).groupBy().avg('hit').collect()

In [ ]:
for name, c in zip (column_names, coefs[0,:]):
    print name, c